# **Projeto AM 2024.2 (Francisco)** - Questão 2

Equipe:
* Beatriz Andrade de Miranda - bam2@cin.ufpe.br
* Camila Siqueira Lins - csl2@cin.ufpe.br
* Luisa Cavalcante - lncc@cin.ufpe.br
* Nicolly Lira Albuquerque - nla@cin.ufpe.br


## Sumário

* [Questão 2a](#scrollTo=1369ntQFYBe8)
* [Questão 2b](#scrollTo=-YUFx3-JcRwg)
* [Questão 2c](#scrollTo=WsuUxlTWcShS)
* [Questão 2d](#scrollTo=O6Me_xWIcS1t)

In [ ]:
!pip install ucimlrepo

In [ ]:
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample

In [ ]:
# Dataset SPECTF heart
spectf_heart = fetch_ucirepo(id=96)

X = spectf_heart.data.features
y = spectf_heart.data.targets.values.ravel()

In [ ]:
print(spectf_heart.metadata)

{'uci_id': 96, 'name': 'SPECTF Heart', 'repository_url': 'https://archive.ics.uci.edu/dataset/96/spectf+heart', 'data_url': 'https://archive.ics.uci.edu/static/public/96/data.csv', 'abstract': 'Data on cardiac Single Proton Emission Computed Tomography (SPECT) images. Each patient classified into two categories: normal and abnormal.', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 267, 'num_features': 44, 'feature_types': ['Integer'], 'demographics': [], 'target_col': ['diagnosis'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2001, 'last_updated': 'Tue Mar 19 2024', 'dataset_doi': '10.24432/C5N015', 'creators': ['Krzysztof Cios', 'Lukasz Kurgan', 'Lucy Goodenday'], 'intro_paper': None, 'additional_info': {'summary': "The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into t

In [ ]:
print(spectf_heart.variables)

         name     role     type demographic description units missing_values
0   diagnosis   Target  Integer        None        None  None             no
1         F1R  Feature  Integer        None        None  None             no
2         F1S  Feature  Integer        None        None  None             no
3         F2R  Feature  Integer        None        None  None             no
4         F2S  Feature  Integer        None        None  None             no
5         F3R  Feature  Integer        None        None  None             no
6         F3S  Feature  Integer        None        None  None             no
7         F4R  Feature  Integer        None        None  None             no
8         F4S  Feature  Integer        None        None  None             no
9         F5R  Feature  Integer        None        None  None             no
10        F5S  Feature  Integer        None        None  None             no
11        F6R  Feature  Integer        None        None  None             no

Funções

In [ ]:
# Calcular as métricas
def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    error_rate = 1 - accuracy

    return error_rate, precision, recall, f1

In [ ]:
# Calcular o intervalo de confiança via bootstrap
def bootstrap_ci(clf, X, y, n_iterations=1000, alpha=0.05):
    metrics = []

    for _ in range(n_iterations):
        X_resampled, y_resampled = resample(X, y)
        clf.fit(X_resampled, y_resampled)
        y_pred = clf.predict(X)
        metrics.append(compute_metrics(y, y_pred))

    metrics = np.array(metrics)

    # Para cada métrica (taxa de erro, precisão, recall, F1), calcular o Intervalo de Confiança
    intervalo_confianca = []
    for i in range(metrics.shape[1]):
        ci_lower = np.percentile(metrics[:, i], 100 * (alpha / 2))
        ci_upper = np.percentile(metrics[:, i], 100 * (1 - alpha / 2))
        mean_est = np.mean(metrics[:, i])
        intervalo_confianca.append((mean_est, ci_lower, ci_upper))

    return intervalo_confianca

## Questão 2a

Considere novamente o dataset "SPECTF" com duas classes a priori.

a) Use validação cruzada estratificada “30 × 10-folds” para avaliar e comparar os 5
classificadores:
* i) bayesiano gaussiano
* ii) bayesiano baseado em k-vizinhos
* iii) bayesiano baseado na janela de Parzen
* iv) regressão logística
* v) usando a regra do voto majoritário a partir dos 4 primeiros classificadores

Quando necessario, faça validação cruzada 5-folds nos 9 folds restantes para fazer ajuste de hiper-parametros e depois treine o modelo novamente com o conjunto aprendizagem de 9-folds usando os valores selecionados para os
hiper-parametros. Use amostragem estratificada.

In [ ]:
# Configuração da validação cruzada 30x10-folds
skf = StratifiedKFold(n_splits=10)

In [ ]:
# Inicialização dos classificadores
gnb = GaussianNB()
knn = KNeighborsClassifier()
parzen = KNeighborsClassifier(weights='distance')  # Aproximação da janela de Parzen
logreg = LogisticRegression(max_iter=10000)

In [ ]:
# Classificador de voto majoritário
voting_clf = VotingClassifier(estimators=[('gnb', gnb), ('knn', knn), ('parzen', parzen), ('logreg', logreg)], voting='hard')

In [ ]:
accuracies_gnb, f1_scores_gnb = [], []
accuracies_knn, f1_scores_knn = [], []
accuracies_parzen, f1_scores_parzen = [], []
accuracies_logreg, f1_scores_logreg = [], []
accuracies_voting, f1_scores_voting = [], []

# Validação cruzada
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Ajuste de hiperparâmetros usando GridSearchCV para KNN e Regressão Logística
    param_grid_knn = {'n_neighbors': [3, 5, 7]}
    grid_knn = GridSearchCV(knn, param_grid_knn, cv=5)
    grid_knn.fit(X_train, y_train)
    best_knn = grid_knn.best_estimator_

    param_grid_logreg = {'C': [0.01, 0.1, 1, 10]}
    grid_logreg = GridSearchCV(logreg, param_grid_logreg, cv=5)
    grid_logreg.fit(X_train, y_train)
    best_logreg = grid_logreg.best_estimator_

    # Treinamento
    gnb.fit(X_train, y_train)
    best_knn.fit(X_train, y_train)
    parzen.fit(X_train, y_train)
    best_logreg.fit(X_train, y_train)
    voting_clf.fit(X_train, y_train)

    # Previsões
    y_pred_gnb = gnb.predict(X_test)
    y_pred_knn = best_knn.predict(X_test)
    y_pred_parzen = parzen.predict(X_test)
    y_pred_logreg = best_logreg.predict(X_test)
    y_pred_voting = voting_clf.predict(X_test)

    # Acurácia e F1-score
    accuracies_gnb.append(accuracy_score(y_test, y_pred_gnb))
    f1_scores_gnb.append(f1_score(y_test, y_pred_gnb, average='macro'))

    accuracies_knn.append(accuracy_score(y_test, y_pred_knn))
    f1_scores_knn.append(f1_score(y_test, y_pred_knn, average='macro'))

    accuracies_parzen.append(accuracy_score(y_test, y_pred_parzen))
    f1_scores_parzen.append(f1_score(y_test, y_pred_parzen, average='macro'))

    accuracies_logreg.append(accuracy_score(y_test, y_pred_logreg))
    f1_scores_logreg.append(f1_score(y_test, y_pred_logreg, average='macro'))

    accuracies_voting.append(accuracy_score(y_test, y_pred_voting))
    f1_scores_voting.append(f1_score(y_test, y_pred_voting, average='macro'))

df_metrics = pd.DataFrame({
    'GaussianNB_Acurácia': accuracies_gnb,
    'GaussianNB_F1': f1_scores_gnb,
    'KNN_Acurácia': accuracies_knn,
    'KNN_F1': f1_scores_knn,
    'Parzen_Acurácia': accuracies_parzen,
    'Parzen_F1': f1_scores_parzen,
    'LogReg_Acurácia': accuracies_logreg,
    'LogReg_F1': f1_scores_logreg,
    'Voting_Acurácia': accuracies_voting,
    'Voting_F1': f1_scores_voting
})

print("\nResumo das métricas dos modelos:")
df_metrics


Resumo das métricas dos modelos:


,GaussianNB_Acurácia,GaussianNB_F1,KNN_Acurácia,KNN_F1,Parzen_Acurácia,Parzen_F1,LogReg_Acurácia,LogReg_F1,Voting_Acurácia,Voting_F1
0,0.518519,0.507714,0.814815,0.714588,0.777778,0.678571,0.666667,0.584615,0.666667,0.613672
1,0.703704,0.644737,0.925926,0.877273,0.777778,0.559783,0.925926,0.877273,0.851852,0.785714
2,0.925926,0.903571,0.888889,0.800000,0.888889,0.828753,0.888889,0.800000,0.925926,0.892857
3,0.666667,0.649351,0.629630,0.464286,0.666667,0.486258,0.703704,0.614286,0.740741,0.645403
4,0.740741,0.715789,0.777778,0.678571,0.777778,0.678571,0.740741,0.600423,0.814815,0.746717
5,0.814815,0.785374,0.814815,0.666667,0.777778,0.631818,0.925926,0.877273,0.851852,0.785714
6,0.629630,0.602941,0.666667,0.486258,0.666667,0.486258,0.777778,0.678571,0.666667,0.544090
7,0.500000,0.460925,0.500000,0.390991,0.500000,0.390991,0.538462,0.350000,0.423077,0.344538
8,0.769231,0.706767,0.730769,0.422222,0.730769,0.422222,0.769231,0.556818,0.730769,0.529716
9,0.692308,0.660131,0.769231,0.628571,0.730769,0.641026,0.846154,0.704545,0.730769,0.641026


In [ ]:
df_metrics.to_csv('metricas_questao2a.csv', index=False)

In [ ]:
mean_acc_gnb = np.mean(accuracies_gnb)
mean_acc_knn = np.mean(accuracies_knn)
mean_acc_parzen = np.mean(accuracies_parzen)
mean_acc_logreg = np.mean(accuracies_logreg)
mean_acc_voting = np.mean(accuracies_voting)

mean_f1_gnb = np.mean(f1_scores_gnb)
mean_f1_knn = np.mean(f1_scores_knn)
mean_f1_parzen = np.mean(f1_scores_parzen)
mean_f1_logreg = np.mean(f1_scores_logreg)
mean_f1_voting = np.mean(f1_scores_voting)

print(f"GaussianNB médias - Acurácia: {mean_acc_gnb:.4f}, F1-Score: {mean_f1_gnb:.4f}")
print(f"KNN médias - Acurácia: {mean_acc_knn:.4f}, F1-Score: {mean_f1_knn:.4f}")
print(f"Parzen médias - Acurácia: {mean_acc_parzen:.4f}, F1-Score: {mean_f1_parzen:.4f}")
print(f"Regressão Logística médias - Acurácia: {mean_acc_logreg:.4f}, F1-Score: {mean_f1_logreg:.4f}")
print(f"Voto Majoritário médias - Acurácia: {mean_acc_voting:.4f}, F1-Score: {mean_f1_voting:.4f}")

GaussianNB médias - Acurácia: 0.6962, F1-Score: 0.6637
KNN médias - Acurácia: 0.7519, F1-Score: 0.6129
Parzen médias - Acurácia: 0.7295, F1-Score: 0.5804
Regressão Logística médias - Acurácia: 0.7783, F1-Score: 0.6644
Voto Majoritário médias - Acurácia: 0.7403, F1-Score: 0.6529


* **Regressão Logística** apresentou o melhor equilíbrio entre acurácia e F1-Score.
* **KNN** e **Voto Majoritário** são boas alternativas se priorizarmos acurácia com menos foco na gestão de desbalanceamento por terem boas acurácias, mas com F1-Scores menos consistentes.
* **GaussianNB** pode ser considerado para tarefas simples e quando a independência entre variáveis é uma suposição razoável.
* **Parzen** é menos recomendado quando o equilíbrio entre precisão e cobertura é necessário, dado seu baixo F1-Score.

## Questão 2b

b) Obtenha uma estimativa pontual e um intervalo de confiança para cada metrica
de avaliação do classificadores (Taxa de erro, precisão, cobertura, F-measure);

In [ ]:
classifiers = {
    "GaussianNB": gnb,
    "KNN": knn,
    "Parzen": parzen,
    "Logistic Regression": logreg,
    "Voting Classifier": voting_clf
}

In [ ]:
# Bootstrap para cada classificador e exibir intervalos de confiança
for name, clf in classifiers.items():
    print(f"\n{name}")
    cis = bootstrap_ci(clf, X, y)

    metric_names = ['Taxa de Erro', 'Precisão', 'Cobertura', 'F1-score']
    for i, ci in enumerate(cis):
        print(f"{metric_names[i]}: Estimativa pontual = {ci[0]:.4f}, IC 95% = [{ci[1]:.4f}, {ci[2]:.4f}]")


GaussianNB
Taxa de Erro: Estimativa pontual = 0.2892, IC 95% = [0.2584, 0.3333]
Precisão: Estimativa pontual = 0.8573, IC 95% = [0.8415, 0.8711]
Cobertura: Estimativa pontual = 0.7108, IC 95% = [0.6667, 0.7416]
F1-score: Estimativa pontual = 0.7382, IC 95% = [0.6978, 0.7660]

KNN
Taxa de Erro: Estimativa pontual = 0.1973, IC 95% = [0.1610, 0.2398]
Precisão: Estimativa pontual = 0.8084, IC 95% = [0.7688, 0.8449]
Cobertura: Estimativa pontual = 0.8027, IC 95% = [0.7602, 0.8390]
F1-score: Estimativa pontual = 0.8035, IC 95% = [0.7653, 0.8383]

Parzen
Taxa de Erro: Estimativa pontual = 0.0996, IC 95% = [0.0712, 0.1312]
Precisão: Estimativa pontual = 0.9034, IC 95% = [0.8741, 0.9318]
Cobertura: Estimativa pontual = 0.9004, IC 95% = [0.8688, 0.9288]
F1-score: Estimativa pontual = 0.9008, IC 95% = [0.8705, 0.9311]

Logistic Regression
Taxa de Erro: Estimativa pontual = 0.1116, IC 95% = [0.0787, 0.1461]
Precisão: Estimativa pontual = 0.8904, IC 95% = [0.8572, 0.9219]
Cobertura: Estimativa pon

* **Parzen** se destacou como o melhor modelo em todos os aspectos (taxa de erro, precisão, cobertura e F1-score), seguido de perto pela **Regressão Logística**.
* **GaussianNB** teve o pior desempenho, especialmente em termos de taxa de erro e cobertura.
* **KNN** apresentou um desempenho intermediário, superando o GaussianNB, mas ainda atrás dos modelos Parzen e Regressão Logística.
* **Voting Classifier** obteve resultados sólidos, mas ficou um pouco atrás dos melhores modelos individuais.

## Questão 2c

c) Usar o Friedman test (teste não parametrico) para comparar os classificadores,
e o pós teste (Nemenyi test), usando cada uma das métricas

## Questão 2d

d) Para cada metrica de avaliação, plot a curva de aprendizagem para o
classificador bayesiano Gaussiano. Usando amostragem estratificada, use 70%
dos dados para treinamento e 30% para teste. Treine o algoritmo com conjuntos
de treinamento de 5% a 100% do conjunto original de treinamento, com passo
de 5% (usando amostragem estratificada). Comente.